In [ ]:
from pandas import read_csv
from numpy import array, linspace, zeros, abs, mean, arange, asarray
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fftpack import fft
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
def data_filter(data, data_t, sr, low, high): #a filter for low and high cut-off
    '''
    data: sEMG data in a single channel ( n x 1 array). 
    data_t: transposed sEMG data (1 x n array)
    sr: sampleing rate, same time units as period
    low: Low cut-off frequency (high-pass filter)
    high: High cut-off frequency (low-pass filter)
    return: filtered data
    '''

    low=low
    high=high

    order = 5
    sos = signal.butter(order,(low, high), btype='bandpass', fs=sr, output='sos')
    data_filtered=zeros((data.shape[0]))
    for index, column in enumerate(data_t):
        forward = signal.sosfilt(sos, column)
        backwards = signal.sosfilt(sos, forward[-1::-1])
    L=len(backwards)
    for ind, item in enumerate(backwards):
        data_filtered[L-1]=item
        L-=1
    
    return data_filtered

In [ ]:
# Loading
path='PATH'
Tremor=read_csv(path)
Time_oneline=960*(1/48000) #Sampleing rate in recording: 48KHz
# print(Time_oneline)
Tremor_array=Tremor.to_numpy() # raw exported file in an array
header=Tremor_array[:,0] #header for idenfication of data blocks
Patient_ID=Tremor_array[8,2]

In [ ]:
# Identify the location of data in each channel
detector_LivePlay=list()
i=0
j=0
for idx, item in enumerate(header):
    if item=='LivePlay' and i==0 and j==0:
        i+=1
        j+=1
        detector_LivePlay.append(idx)
    elif item!='LivePlay' and i>0 and j>0:
        detector_LivePlay.append(idx-1)
        i+=1
        j=0
    elif item=='LivePlay' and i!=0 and j==0:
        detector_LivePlay.append(idx)
        i+=1
        j+=1
              

detector_LivePlay.append(len(header)-1)
# print(detector_LivePlay)

In [ ]:
# separate data into each channol
CH1=detector_LivePlay[0:2] #CH1_idx
CH2=detector_LivePlay[2:4] #CH2_idx
CH3=detector_LivePlay[4:6] #CH3_idx
CH4=detector_LivePlay[6:]  #CH4_idx
Channel_set=[CH1,CH2,CH3,CH4]


In [ ]:
# Extracting each channel data and create transposed array
def channel_data(CH_idx, Tremor_array):
    CH_lines=CH_idx[1]-CH_idx[0]+1
    Data_sample_in_each_line=(Tremor_array.shape[1]-2)
    total_number=CH_lines*Tremor_array.shape[1]
    CH_data=zeros(total_number)
    CH_data_t=zeros((1,total_number))
    counter=0
    for counter_line in range(CH_idx[0],CH_idx[1]+1,1):
        for counter_sample in range(3,Tremor_array.shape[1],1):
            CH_data[counter]=Tremor_array[counter_line][counter_sample]
            CH_data_t[0][counter]=Tremor_array[counter_line][counter_sample]
            # # for abs
            # CH_data[counter]=abs(CH_data[counter])
            # CH_data_t[0][counter]=abs(CH_data_t[0][counter])
            counter+=1

    
    return CH_data, CH_data_t

In [ ]:
#Data preprocessing
Channel_name=['CH1','CH2','CH3','CH4']
fs = 48000   # sample rate, Hz
Low=20
High=300

fig,axes = plt.subplots(figsize=(25,40), ncols=2, nrows=4, sharex=False)

for idx, CH in enumerate(Channel_set):
    #filtering
    ch_data, ch_data_t=channel_data(CH, Tremor_array)
    T = len(ch_data)/fs         # Sample Period
    data_filtered=data_filter(ch_data, ch_data_t, fs, Low, High)
    t = linspace(0, T, len(data_filtered), endpoint=False)
    axes[idx][0].plot(t, ch_data, 'b-', label='raw signal');
    axes[idx][0].plot(t, data_filtered, 'r-', linewidth=2, label='filtered signal (20-300Hz)');
    axes[idx][0].set_xlabel('Time [s]',fontsize=20);
    axes[idx][0].set_ylabel('amplitude (uV)',fontsize=20);
    axes[idx][0].set_aspect('auto')
    axes[idx][0].minorticks_on()
    axes[idx][0].grid(which = 'major', axis = 'both')
    axes[idx][0].xaxis.set_tick_params(rotation = 0, labelsize = 12, colors = 'k')
    axes[idx][0].set_title('Butterworth bandpass filtering of 'f'{Channel_name[idx]}', fontsize=20);
    axes[idx][0].legend(loc='best');
    axes[idx][0].grid()

    #downsamping to 1000Hz
    re_sr=1000
    y= data_filtered
    samps = round(T*re_sr)
    preprocessed_data = signal.resample(y, samps)

    #rectifying
    resampled_data=zeros((len(preprocessed_data)))
    for idx2,item2 in enumerate(preprocessed_data):
        resampled_data[idx2]=abs(item2)
    t = linspace(0, T, len(data_filtered), endpoint=False)

    T_resampled = len(resampled_data)/re_sr
    t_resampled = linspace(0, T, len(resampled_data), endpoint=False)
    
    #smoothing
    alpha = 0.2  # Smoothing parameter (0 < alpha < 1)
    model = ExponentialSmoothing(asarray(resampled_data))
    result = model.fit(smoothing_level=alpha, optimized=False)
    # Extract smoothed data
    smoothed_data = result.fittedvalues
    # plt.plot(smoothed_data)

    #fft
    data_back_fft=fft(smoothed_data, axis=0)
    rows = smoothed_data.shape[0]
    freqs=(re_sr/2)*linspace(0,1,int(rows/2))
    amplitudes_back = (2.0/rows)* abs(data_back_fft[:rows//2])

    axes[idx][1].plot(freqs,amplitudes_back)
    axes[idx][1].set_xlim(1,30)
    axes[idx][1].set_ylabel('Amplitude (uV)',fontsize=20)
    axes[idx][1].set_xlabel('Frequency (Hz)',fontsize=20)
    axes[idx][1].set_aspect('auto')
    axes[idx][1].minorticks_on()
    axes[idx][1].grid(which = 'major', axis = 'both')
    axes[idx][1].xaxis.set_tick_params(rotation = 0, labelsize = 12, colors = 'k')

    axes[idx][1].xaxis.set_ticks(arange(1, 30, 1))
    axes[idx][1].set_title('ID:'f'{Patient_ID}''-Power spectrum of 'f'{Channel_name[idx]}', fontsize=20);


plt.savefig(f'{Patient_ID}''-Power spectrum-State.png');
plt.show();